In [1]:
!python ../src/utils.py
%reload_ext autoreload

# Mass Reconstruction for Each dE/dx Estimator

This notebook reconstructs the mass for each track using different dE/dx estimators (harmonic mean, truncated mean, etc.) from AOD data. The mass is calculated using the formula:

$$ I_h = K \frac{m^2}{p^2} + C $$

where:
- $I_h$ is the estimator mean for each track,
- $p$ is the track momentum (for high-energy tracks, $p \approx p_T$),
- $K = 2.49$ and $C = 3.18$ (from 2018 simulation).

For each estimator, we solve for $m$:

$$ m = p \cdot \sqrt{\frac{I_h - C}{K}} $$

Each cell below reconstructs and plots the mass distribution for a different estimator.


In [2]:
# Import all required libraries
import numpy as np
import os
import matplotlib.pyplot as plt
import ROOT as rt
import pandas as pd
from ROOT import VecOps
import utils

# Enable JSROOT 
%jsroot on

from analysis import df , HMNCSBR, TRUNCSBR

print("cwd:", os.getcwd())

cwd: /home/bothsides/projects/optimizing_DEDx_estimator/notebooks


In [3]:
from anal import build_df_final

INPUT = "../data/HSCPgluino_M-1800_fromAOD.root"
TREE  = "HSCPFullAODAnalyzer/Events"

In [4]:
df_final, rows, mask = build_df_final(INPUT, TREE, weight_branch=None)
print("Events after full selection:", df_final.Count().GetValue())

Events after full selection: 558


In [5]:
df_filtered = df_final # Filtered DataFrame after applying all cuts

In [6]:
# Constants from 2018 simulation
K = 2.49
C = 3.18

# Load and prepare data
columns_needed = ['IsoTrack_p'] + HMNCSBR + TRUNCSBR
df_pd = df_filtered.AsNumpy(columns_needed)

print(f"Constants: K = {K}, C = {C}\n")
print(f"Available estimators:")
print(f"Harmonic means: {HMNCSBR}")
print(f"Truncated means: {TRUNCSBR}")
print(f"Number of tracks: {len(df_pd['IsoTrack_p'])}")
print(columns_needed)


# Quick peek at the data
print("\nKeys:", list(df_pd.keys()))


Constants: K = 2.49, C = 3.18

Available estimators:
Harmonic means: ['DeDx_IhStrip1', 'DeDx_IhStrip', 'DeDx_IhStrip3', 'DeDx_IhStrip4']
Truncated means: ['DeDx_ItStrip0', 'DeDx_ItStrip5', 'DeDx_ItStrip10', 'DeDx_ItStrip15', 'DeDx_ItStrip20', 'DeDx_ItStrip25', 'DeDx_ItStrip30', 'DeDx_ItStrip35', 'DeDx_ItStrip']
Number of tracks: 558
['IsoTrack_p', 'DeDx_IhStrip1', 'DeDx_IhStrip', 'DeDx_IhStrip3', 'DeDx_IhStrip4', 'DeDx_ItStrip0', 'DeDx_ItStrip5', 'DeDx_ItStrip10', 'DeDx_ItStrip15', 'DeDx_ItStrip20', 'DeDx_ItStrip25', 'DeDx_ItStrip30', 'DeDx_ItStrip35', 'DeDx_ItStrip']

Keys: ['IsoTrack_p', 'DeDx_IhStrip1', 'DeDx_IhStrip', 'DeDx_IhStrip3', 'DeDx_IhStrip4', 'DeDx_ItStrip0', 'DeDx_ItStrip5', 'DeDx_ItStrip10', 'DeDx_ItStrip15', 'DeDx_ItStrip20', 'DeDx_ItStrip25', 'DeDx_ItStrip30', 'DeDx_ItStrip35', 'DeDx_ItStrip']


In [7]:
any_col = columns_needed[0]
n_events = df_pd[any_col].shape[0]
print("Number of events passing all cuts:", n_events)

Number of events passing all cuts: 558


In [8]:
pt_lists     = df_pd["IsoTrack_p"]         # array of python lists
n_tracks_arr = np.array([ len(lst) for lst in pt_lists ])

print("Number of events:", pt_lists.shape[0])
total_tracks = n_tracks_arr.sum()
print("Total tracks in all events:", total_tracks)
print("Tracks per event array:\n", n_tracks_arr)

Number of events: 558
Total tracks in all events: 1252
Tracks per event array:
 [ 2  2  1  2  2  2  1  3  6  2  4  1  3  3  2  5  2  2  2  2  3  3  2  4
  2  2  2  2  3  3  5  2  1  2  3  2  2  4  2  1  1  2  2  2  1  1  5  2
  3  3  2  2  1  2  2  1  2  3  2  2  2  1  2  2  2  2  2  2  2  1  2  2
  1  6  3  3  3  2  4  3  1  1  3  3  2  2  4  4  1  2  2  3  4  1  2  1
  2  1  2  1  2  2  2  6  2  1  2  2  1  3  2  3  3  3  1  2  7  2  3  1
  1  2  2  2  1  3  3  2  4  1  2  1  1  2  2  1  2  1  2  2  1  3  2  2
  2  2  2  3  2  2  2  2  2  2  2  2  1  1  2  2  1  4  3  2  2  2  1  2
  2  1  2  3  1  1  1  4  4  2  1  1  2  2  1  2  1  3  2  1  2  3  3  1
  1  7  2  1  2  1  1  2  2  3  1  5  2  1  1  1  1  2  2  4  2  2  3  2
  1  3  2  4  1  1  4  2  4  2  2  2  1  2  5  3  2  3  1  1  2  1  1  3
  2  6  1  4  4  3  1  3  1  2  2  3  2  3  1 10  2  6  1  1  6  2  1  1
  2  2  3  2  2  2  2  2  1  2  4  3  2  2  2  1  2  3  2  1  1  2  2  2
  1  1  1  2  1  3  2  2  3  2  2  1  2  2  

In [9]:
# # Inspect df_pd structure and content
# print("=== df_pd Structure ===")
# print(f"Type: {type(df_pd)}")
# print(f"Keys (columns): {list(df_pd.keys())}")

# print("\n=== Shape and Types ===")
# for key in df_pd.keys():
#     data = df_pd[key]
#     print(f"{key}: shape={np.array(data).shape}, type={type(data)}")
    
#     # Show first few values
#     if hasattr(data, '__len__') and len(data) > 0:
#         try:
#             sample = data[:5] if len(data) > 5 else data
#             print(f"  First few values: {sample}")
#         except:
#             print(f"  Sample value: {data[0] if len(data) > 0 else 'No data'}")
#     print()

# print("\n=== Alternative: Use pandas for better display ===")


# # Convert to pandas DataFrame for better visualization
# try:
#     # Try to convert to pandas - handle potential nested arrays
#     df_display = {}
#     for key, values in df_pd.items():
#         # Take first 10 entries for display
#         if hasattr(values, '__len__') and len(values) > 0:
#             df_display[key] = values[:10]
#         else:
#             df_display[key] = values
    
#     pdf = pd.DataFrame(df_display)
#     print("First 10 rows as pandas DataFrame:")
#     print(pdf)
# except Exception as e:
#     print(f"Could not convert to pandas DataFrame: {e}")
    
#     # Fallback: show raw data structure
#     print("\nRaw data preview:")
#     for key in list(df_pd.keys())[:3]:  # Show first 3 columns
#         print(f"\n{key}:")
#         data = df_pd[key]
#         if hasattr(data, '__len__') and len(data) > 0:
#             print(f"  Length: {len(data)}")
#             print(f"  First 5 values: {data[:5]}")
#         else:
#             print(f"  Value: {data}")

In [10]:
#sanity‑check on if the arrays have consistent shapes

keys = [
    "IsoTrack_p",
    "DeDx_IhStrip1", "DeDx_IhStrip", "DeDx_IhStrip3", "DeDx_IhStrip4",
    "DeDx_ItStrip0", "DeDx_ItStrip5", "DeDx_ItStrip10", "DeDx_ItStrip15",
    "DeDx_ItStrip20", "DeDx_ItStrip25", "DeDx_ItStrip30", "DeDx_ItStrip35",
    "DeDx_ItStrip",
]

ok, bad_rows = utils.check_branch_shapes(df_pd, keys)
if ok:
    print("All arrays have consistent shapes.")
else:
    print(f"Arrays have inconsistent shapes. Bad rows: {bad_rows}")
    print("Check the data for potential issues.")

        
        

All rows consistent? True
All arrays have consistent shapes.


In [11]:
# Extract actual values from ROOT data structures
print("=== Extracting Actual Values ===")

# Method 1: Use VecOps to convert ROOT vectors to Python lists
print("\n--- Using VecOps conversion ---")
for key in list(df_pd.keys())[:]:  # Show all the columns
    data = df_pd[key]
    print(f"\n{key}:")
    print(f"  Raw data type: {type(data)}")
    print(f"  Length: {len(data)}")
    
    # Convert first few entries to see actual values
    actual_values = []
    for i in range(min(5, len(data))):
        try:
            # Try to convert using VecOps
            if hasattr(data[i], '__len__'):  # If it's array-like
                val = list(data[i])  # Convert to Python list
            else:
                val = data[i]  # Already a scalar
            actual_values.append(val)
        except Exception as e:
            actual_values.append(f"Error: {e}")
    
    print(f"  First 5 actual values: {actual_values}")



=== Extracting Actual Values ===

--- Using VecOps conversion ---

IsoTrack_p:
  Raw data type: <class 'ROOT._pythonization._rdf_utils.ndarray'>
  Length: 558
  First 5 actual values: [[1607.3076766502218, 33.846389342860995], [1645.7572310559751, 1465.1951680931663], [1733.4500116273564], [2365.7514039063904, 1903.1668794635832], [1701.8068304732906, 1008.9511473400491]]

DeDx_IhStrip1:
  Raw data type: <class 'ROOT._pythonization._rdf_utils.ndarray'>
  Length: 558
  First 5 actual values: [[6.2467265129089355, 3.4969427585601807], [5.511447906494141, 5.713008403778076], [5.352411270141602], [6.777451992034912, 7.3498711585998535], [4.99361515045166, 6.76262092590332]]

DeDx_IhStrip:
  Raw data type: <class 'ROOT._pythonization._rdf_utils.ndarray'>
  Length: 558
  First 5 actual values: [[6.19785213470459, 3.3484208583831787], [5.4514031410217285, 5.572141170501709], [5.29805850982666], [6.663329124450684, 7.245083332061768], [4.9868388175964355, 6.7018561363220215]]

DeDx_IhStrip3:
 

## Harmonic Mean Estimator Mass Reconstruction

In [12]:
keys_to_flatten = [
    "IsoTrack_p",
    "DeDx_IhStrip1", "DeDx_IhStrip", "DeDx_IhStrip3", "DeDx_IhStrip4",
    "DeDx_ItStrip0", "DeDx_ItStrip5", "DeDx_ItStrip10", "DeDx_ItStrip15",
    "DeDx_ItStrip20", "DeDx_ItStrip25", "DeDx_ItStrip30", "DeDx_ItStrip35",
    "DeDx_ItStrip",
]

# Flatten the branches
flattened_data = utils.flatten_branches(df_pd, keys_to_flatten, drop_sentinel=None)
print("Branches flattened successfully.")

IsoTrack_p: shape=(1252,), range=9.895–458320.549
DeDx_IhStrip1: shape=(1252,), range=-1.000–21.198
DeDx_IhStrip: shape=(1252,), range=-1.000–21.065
DeDx_IhStrip3: shape=(1252,), range=-1.000–20.930
DeDx_IhStrip4: shape=(1252,), range=-1.000–20.795
DeDx_ItStrip0: shape=(1252,), range=-1.000–21.455
DeDx_ItStrip5: shape=(1252,), range=-1.000–21.455
DeDx_ItStrip10: shape=(1252,), range=-1.000–21.455
DeDx_ItStrip15: shape=(1252,), range=-1.000–21.455
DeDx_ItStrip20: shape=(1252,), range=-1.000–20.879
DeDx_ItStrip25: shape=(1252,), range=-1.000–20.879
DeDx_ItStrip30: shape=(1252,), range=-1.000–20.879
DeDx_ItStrip35: shape=(1252,), range=-1.000–20.879
DeDx_ItStrip: shape=(1252,), range=-1.000–20.111
Branches flattened successfully.


In [13]:
# Mass reconstruction for harmonic2 mean estimator (DeDx_IhStrip)

print("\n=== Mass Reconstruction for Harmonic2 Mean Estimator (DeDx_IhStrip) ===")

# getting pt and IhStrip data
pt_data = df_pd['IsoTrack_p']
ih_data = df_pd['DeDx_IhStrip']

# Convert to numpy and flatten if needed
if hasattr(pt_data, '__len__') and len(pt_data) > 0: # check if pt_data is iterable
    # Flatten and convert to numpy arrays
    pt = np.concatenate([np.array(event) if hasattr(event, '__iter__') else [event] for event in pt_data])
    Ih_harmonic2 = np.concatenate([np.array(event) if hasattr(event, '__iter__') else [event] for event in ih_data])
else:
    pt = np.array(pt_data)
    Ih_harmonic2 = np.array(ih_data)


# Remove Ih <= 0 (and keep pt aligned)
pos = Ih_harmonic2 > 0
pt = pt[pos]
Ih_harmonic2 = Ih_harmonic2[pos]

print(f"Data shape - p: {pt.shape}, Ih: {Ih_harmonic2.shape}")
print(f"pt range: {np.min(pt):.3f} to {np.max(pt):.3f} GeV")
print(f"Ih range: {np.min(Ih_harmonic2):.3f} to {np.max(Ih_harmonic2):.3f} MeV/cm")

# Only use tracks where Ih > C to avoid sqrt of negative
mask = Ih_harmonic2 > C
print(f"Tracks with Ih > C: {np.sum(mask)} / {len(Ih_harmonic2)}")

mass_harmonic = np.zeros_like(pt)
mass_harmonic[mask] = pt[mask] * np.sqrt((Ih_harmonic2[mask] - C) / K)

print(f"Mass range: {np.min(mass_harmonic[mask]):.3f} - {np.max(mass_harmonic[mask]):.3f} GeV")

# Create ROOT histogram for inline display with JSROOT
h_mass_harmonic = rt.TH1D("h_mass_harmonic", 
                           "Mass Distribution (Harmonic Mean Estimator - DeDx_IhStrip);Reconstructed Mass [GeV];Number of Tracks",
                           100, 0, 5000)

# Fill histogram
for mass in mass_harmonic[mask]:
    h_mass_harmonic.Fill(mass)

# Style the histogram
h_mass_harmonic.SetFillColor(rt.kBlue)
h_mass_harmonic.SetFillStyle(1001)
h_mass_harmonic.SetLineColor(rt.kBlue+2)
h_mass_harmonic.SetLineWidth(2)

# Create canvas and draw histogram
canvas_harmonic = rt.TCanvas("canvas_harmonic", "Harmonic Mean Mass Distribution", 800, 600)
h_mass_harmonic.Draw("HIST")
canvas_harmonic.Draw()

print(f"Harmonic Mean (DeDx_IhStrip): {len(mass_harmonic[mask])} tracks with valid mass reconstruction")
# print(f"Mean mass: {np.mean(mass_harmonic[mask]):.1f} GeV")
# print(f"Median mass: {np.median(mass_harmonic[mask]):.1f} GeV")


=== Mass Reconstruction for Harmonic2 Mean Estimator (DeDx_IhStrip) ===
Data shape - p: (1187,), Ih: (1187,)
pt range: 9.895 to 458320.549 GeV
Ih range: 0.764 to 21.065 MeV/cm
Tracks with Ih > C: 1113 / 1187
Mass range: 0.524 - 479212.671 GeV
Harmonic Mean (DeDx_IhStrip): 1113 tracks with valid mass reconstruction


## Truncated Mean Estimator Mass Reconstruction

In [14]:
# Mass reconstruction for Default Truncated Mean Estimator (DeDx_ItStrip)
print("\n=== Mass Reconstruction for 40% Truncated Mean Estimator (DeDx_ItStrip) ===")

# Get the default truncated mean estimator data
pt_data_default = df_pd['IsoTrack_p']
it_data_default = df_pd['DeDx_ItStrip']

# Convert and flatten the data properly (handle nested arrays)
try:
    pt_default = np.concatenate([np.array(track_pt) if hasattr(track_pt, '__iter__') else [track_pt] for track_pt in pt_data_default])
    It_default = np.concatenate([np.array(track_ih) if hasattr(track_ih, '__iter__') else [track_ih] for track_ih in it_data_default])
except:
    # Fallback to direct conversion if data is already flat
    pt_default = np.array(pt_data_default)
    It_default = np.array(it_data_default)


# Remove It <= 0 (and keep pt aligned)
pos = It_default > 0
pt_default = pt_default[pos]
It_default = It_default[pos]

print(f"Data shape - p: {pt_default.shape}, Ih: {It_default.shape}")
print(f"pt range: {pt_default.min():.3f} - {pt_default.max():.3f} GeV")
print(f"Ih range: {It_default.min():.3f} - {It_default.max():.3f}")

# Apply mask for valid Ih values (Ih > C)
mask_default = It_default > C
print(f"Tracks with Ih > C: {np.sum(mask_default)} / {len(It_default)}")

# Mass reconstruction using the formula: m = p * sqrt((Ih - C) / K)
mass_default = pt_default[mask_default] * np.sqrt((It_default[mask_default] - C) / K)

print(f"Mass range: {mass_default.min():.3f} - {mass_default.max():.3f} GeV")

# Create ROOT histogram for inline display with JSROOT
hist_default = rt.TH1D("hist_default", "Mass Distribution (Default Truncated Mean Estimator - DeDx_ItStrip);Reconstructed Mass [GeV];Number of Tracks", 100, 0, 5000)

# Fill histogram
for mass in mass_default:
    hist_default.Fill(mass)

# Style and draw histogram
hist_default.SetFillColor(rt.kOrange)
hist_default.SetFillStyle(1001)

# Create canvas and draw histogram
canvas_default = rt.TCanvas("canvas_default", "Default Truncated Mean Mass Distribution", 800, 600)
hist_default.Draw("HIST")
canvas_default.Draw()

print(f"Default Truncated Mean (DeDx_ItStrip): {np.sum(mask_default)} tracks with valid mass reconstruction")
# print(f"Mean mass: {mass_default.mean():.1f} GeV")
# print(f"Median mass: {np.median(mass_default):.1f} GeV")


=== Mass Reconstruction for 40% Truncated Mean Estimator (DeDx_ItStrip) ===
Data shape - p: (1187,), Ih: (1187,)
pt range: 9.895 - 458320.549 GeV
Ih range: 0.764 - 20.111
Tracks with Ih > C: 973 / 1187
Mass range: 3.305 - 437435.885 GeV
Default Truncated Mean (DeDx_ItStrip): 973 tracks with valid mass reconstruction


## Other Harmonic Mean Estimators

In [16]:
# Mass reconstruction for Harmonic Mean Estimator (DeDx_IhStrip1)

# Get the harmonic mean estimator data
pt_data_h1 = df_pd['IsoTrack_p']
ih_data_h1 = df_pd['DeDx_IhStrip1']

# Convert and flatten the data properly (handle nested arrays)
try:
    pt_h1 = np.concatenate([np.array(track_pt) if hasattr(track_pt, '__iter__') else [track_pt] for track_pt in pt_data_h1])
    Ih_h1 = np.concatenate([np.array(track_ih) if hasattr(track_ih, '__iter__') else [track_ih] for track_ih in ih_data_h1])
except:
    # Fallback to direct conversion if data is already flat
    pt_h1 = np.array(pt_data_h1)
    Ih_h1 = np.array(ih_data_h1)

print(f"Data shape - pt: {pt_h1.shape}, Ih: {Ih_h1.shape}")
print(f"pt range: {pt_h1.min():.3f} - {pt_h1.max():.3f} GeV")
print(f"Ih range: {Ih_h1.min():.3f} - {Ih_h1.max():.3f}")

# Apply mask for valid Ih values (Ih > C)
mask_h1 = Ih_h1 > C
print(f"Tracks with Ih > C: {np.sum(mask_h1)} / {len(Ih_h1)}")

# Mass reconstruction using the formula: m = p * sqrt((Ih - C) / K)
mass_h1 = pt_h1[mask_h1] * np.sqrt((Ih_h1[mask_h1] - C) / K)

print(f"Mass range: {mass_h1.min():.3f} - {mass_h1.max():.3f} GeV")

# Create ROOT histogram for inline display with JSROOT
hist_h1 = rt.TH1D("hist_harm1", "Mass Distribution (Harmonic Mean Estimator - DeDx_IhStrip1);Reconstructed Mass [GeV];Number of Tracks", 100, 0, 5000)

# Fill histogram
for mass in mass_h1:
    hist_h1.Fill(mass)

# Style and draw histogram
hist_h1.SetFillColor(rt.kBlue)
hist_h1.SetFillStyle(1001)

# Create canvas and draw histogram
canvas_h1 = rt.TCanvas("canvas_h1", "Harmonic Mean 1 Mass Distribution", 800, 600)
hist_h1.Draw("HIST")
canvas_h1.Draw()

print(f"Harmonic Mean (DeDx_IhStrip1): {np.sum(mask_h1)} tracks with valid mass reconstruction")
print(f"Mean mass: {mass_h1.mean():.1f} GeV")
print(f"Median mass: {np.median(mass_h1):.1f} GeV")

Data shape - pt: (1252,), Ih: (1252,)
pt range: 9.895 - 458320.549 GeV
Ih range: -1.000 - 21.198
Tracks with Ih > C: 1141 / 1252
Mass range: 2.329 - 485860.350 GeV
Harmonic Mean (DeDx_IhStrip1): 1141 tracks with valid mass reconstruction
Mean mass: 1930.3 GeV
Median mass: 1451.1 GeV


In [17]:
# Mass reconstruction for Harmonic Mean Estimator (DeDx_IhStrip3)

# Get the harmonic mean estimator data
pt_data_h3 = df_pd['IsoTrack_p']
ih_data_h3 = df_pd['DeDx_IhStrip3']

# Convert and flatten the data properly (handle nested arrays)
try:
    pt_h3 = np.concatenate([np.array(track_pt) if hasattr(track_pt, '__iter__') else [track_pt] for track_pt in pt_data_h3])
    Ih_h3 = np.concatenate([np.array(track_ih) if hasattr(track_ih, '__iter__') else [track_ih] for track_ih in ih_data_h3])
except:
    # Fallback to direct conversion if data is already flat
    pt_h3 = np.array(pt_data_h3)
    Ih_h3 = np.array(ih_data_h3)

print(f"Data shape - pt: {pt_h3.shape}, Ih: {Ih_h3.shape}")
print(f"pt range: {pt_h3.min():.3f} - {pt_h3.max():.3f} GeV")
print(f"Ih range: {Ih_h3.min():.3f} - {Ih_h3.max():.3f}")

# Apply mask for valid Ih values (Ih > C)
mask_h3 = Ih_h3 > C
print(f"Tracks with Ih > C: {np.sum(mask_h3)} / {len(Ih_h3)}")

# Mass reconstruction using the formula: m = p * sqrt((Ih - C) / K)
mass_h3 = pt_h3[mask_h3] * np.sqrt((Ih_h3[mask_h3] - C) / K)

print(f"Mass range: {mass_h3.min():.3f} - {mass_h3.max():.3f} GeV")

# Create ROOT histogram for inline display with JSROOT
hist_h3 = rt.TH1D("hist_harm3", "Mass Distribution (Harmonic Mean Estimator - DeDx_IhStrip3);Reconstructed Mass [GeV];Number of Tracks", 100, 0, 5000)

# Fill histogram
for mass in mass_h3:
    hist_h3.Fill(mass)

# Style and draw histogram
hist_h3.SetFillColor(rt.kMagenta)
hist_h3.SetFillStyle(1001)

# Create canvas and draw histogram
canvas_h3 = rt.TCanvas("canvas_h3", "Harmonic Mean 3 Mass Distribution", 800, 600)
hist_h3.Draw("HIST")
canvas_h3.Draw()

print(f"Harmonic Mean (DeDx_IhStrip3): {np.sum(mask_h3)} tracks with valid mass reconstruction")
print(f"Mean mass: {mass_h3.mean():.1f} GeV")
print(f"Median mass: {np.median(mass_h3):.1f} GeV")

Data shape - pt: (1252,), Ih: (1252,)
pt range: 9.895 - 458320.549 GeV
Ih range: -1.000 - 20.930
Tracks with Ih > C: 1067 / 1252
Mass range: 2.803 - 473334.759 GeV
Harmonic Mean (DeDx_IhStrip3): 1067 tracks with valid mass reconstruction
Mean mass: 1963.5 GeV
Median mass: 1430.5 GeV


In [18]:
# Mass reconstruction for Harmonic Mean Estimator (DeDx_IhStrip4)

# Get the harmonic mean estimator data
pt_data_h4 = df_pd['IsoTrack_p']
ih_data_h4 = df_pd['DeDx_IhStrip4']

# Convert and flatten the data properly (handle nested arrays)
try:
    pt_h4 = np.concatenate([np.array(track_pt) if hasattr(track_pt, '__iter__') else [track_pt] for track_pt in pt_data_h4])
    Ih_h4 = np.concatenate([np.array(track_ih) if hasattr(track_ih, '__iter__') else [track_ih] for track_ih in ih_data_h4])
except:
    # Fallback to direct conversion if data is already flat
    pt_h4 = np.array(pt_data_h4)
    Ih_h4 = np.array(ih_data_h4)

print(f"Data shape - pt: {pt_h4.shape}, Ih: {Ih_h4.shape}")
print(f"pt range: {pt_h4.min():.3f} - {pt_h4.max():.3f} GeV")
print(f"Ih range: {Ih_h4.min():.3f} - {Ih_h4.max():.3f}")

# Apply mask for valid Ih values (Ih > C)
mask_h4 = Ih_h4 > C
print(f"Tracks with Ih > C: {np.sum(mask_h4)} / {len(Ih_h4)}")

# Mass reconstruction using the formula: m = p * sqrt((Ih - C) / K)
mass_h4 = pt_h4[mask_h4] * np.sqrt((Ih_h4[mask_h4] - C) / K)

print(f"Mass range: {mass_h4.min():.3f} - {mass_h4.max():.3f} GeV")

# Create ROOT histogram for inline display with JSROOT
hist_h4 = rt.TH1D("hist_harm4", "Mass Distribution (Harmonic Mean Estimator - DeDx_IhStrip4);Reconstructed Mass [GeV];Number of Tracks", 100, 0, 5000)

# Fill histogram
for mass in mass_h4:
    hist_h4.Fill(mass)

# Style and draw histogram
hist_h4.SetFillColor(rt.kCyan)
hist_h4.SetFillStyle(1001)

# Create canvas and draw histogram
canvas_h4 = rt.TCanvas("canvas_h4", "Harmonic Mean 4 Mass Distribution", 800, 600)
hist_h4.Draw("HIST")
canvas_h4.Draw()

print(f"Harmonic Mean (DeDx_IhStrip4): {np.sum(mask_h4)} tracks with valid mass reconstruction")
print(f"Mean mass: {mass_h4.mean():.1f} GeV")
print(f"Median mass: {np.median(mass_h4):.1f} GeV")

Data shape - pt: (1252,), Ih: (1252,)
pt range: 9.895 - 458320.549 GeV
Ih range: -1.000 - 20.795
Tracks with Ih > C: 1021 / 1252
Mass range: 2.613 - 468172.088 GeV
Harmonic Mean (DeDx_IhStrip4): 1021 tracks with valid mass reconstruction
Mean mass: 2007.4 GeV
Median mass: 1428.0 GeV


## Other Truncated Mean Estimators

In [19]:
# Mass reconstruction for Truncated Mean Estimator (DeDx_ItStrip0)



print("\n=== Mass Reconstruction for arithmetic Mean Estimator (DeDx_ItStrip0) ===")
# Get the truncated mean estimator data
pt_data_t0 = df_pd['IsoTrack_p']
ih_data_t0 = df_pd['DeDx_ItStrip0']

# Convert and flatten the data properly (handle nested arrays)
try:
    pt_t0 = np.concatenate([np.array(track_pt) if hasattr(track_pt, '__iter__') else [track_pt] for track_pt in pt_data_t0])
    Ih_t0 = np.concatenate([np.array(track_ih) if hasattr(track_ih, '__iter__') else [track_ih] for track_ih in ih_data_t0])
except:
    # Fallback to direct conversion if data is already flat
    pt_t0 = np.array(pt_data_t0)
    Ih_t0 = np.array(ih_data_t0)

print(f"Data shape - pt: {pt_t0.shape}, Ih: {Ih_t0.shape}")
print(f"pt range: {pt_t0.min():.3f} - {pt_t0.max():.3f} GeV")
print(f"Ih range: {Ih_t0.min():.3f} - {Ih_t0.max():.3f}")

# Apply mask for valid Ih values (Ih > C)
mask_t0 = Ih_t0 > C
print(f"Tracks with Ih > C: {np.sum(mask_t0)} / {len(Ih_t0)}")

# Mass reconstruction using the formula: m = p * sqrt((Ih - C) / K)
mass_t0 = pt_t0[mask_t0] * np.sqrt((Ih_t0[mask_t0] - C) / K)

print(f"Mass range: {mass_t0.min():.3f} - {mass_t0.max():.3f} GeV")

# Create ROOT histogram for inline display with JSROOT
hist_t0 = rt.TH1D("hist_trunc0", "Mass Distribution arithmetic mean;Reconstructed Mass [GeV];Number of Tracks", 100, 0, 5000)

# Fill histogram
for mass in mass_t0:
    hist_t0.Fill(mass)

# Style and draw histogram
hist_t0.SetFillColor(rt.kRed)
hist_t0.SetFillStyle(1001)

# Create canvas and draw histogram
canvas_t0 = rt.TCanvas("canvas_t0", "Truncated Mean 0 Mass Distribution", 800, 600)
hist_t0.Draw("HIST")
canvas_t0.Draw()

print(f"Arithmetic Mean (DeDx_ItStrip0): {np.sum(mask_t0)} tracks with valid mass reconstruction")
print(f"Mean mass: {mass_t0.mean():.1f} GeV")
print(f"Median mass: {np.median(mass_t0):.1f} GeV")


=== Mass Reconstruction for arithmetic Mean Estimator (DeDx_ItStrip0) ===
Data shape - pt: (1252,), Ih: (1252,)
pt range: 9.895 - 458320.549 GeV
Ih range: -1.000 - 21.455
Tracks with Ih > C: 1161 / 1252
Mass range: 2.307 - 501442.611 GeV
Arithmetic Mean (DeDx_ItStrip0): 1161 tracks with valid mass reconstruction
Mean mass: 2010.2 GeV
Median mass: 1532.3 GeV


In [20]:
# Mass reconstruction for Truncated Mean Estimator (DeDx_ItStrip5)

# Get the truncated mean estimator data
pt_data_t5 = df_pd['IsoTrack_p']
ih_data_t5 = df_pd['DeDx_ItStrip5']

# Convert and flatten the data properly (handle nested arrays)
try:
    pt_t5 = np.concatenate([np.array(track_pt) if hasattr(track_pt, '__iter__') else [track_pt] for track_pt in pt_data_t5])
    Ih_t5 = np.concatenate([np.array(track_ih) if hasattr(track_ih, '__iter__') else [track_ih] for track_ih in ih_data_t5])
except:
    # Fallback to direct conversion if data is already flat
    pt_t5 = np.array(pt_data_t5)
    Ih_t5 = np.array(ih_data_t5)

print(f"Data shape - pt: {pt_t5.shape}, Ih: {Ih_t5.shape}")
print(f"pt range: {pt_t5.min():.3f} - {pt_t5.max():.3f} GeV")
print(f"Ih range: {Ih_t5.min():.3f} - {Ih_t5.max():.3f}")

# Apply mask for valid Ih values (Ih > C)
mask_t5 = Ih_t5 > C
print(f"Tracks with Ih > C: {np.sum(mask_t5)} / {len(Ih_t5)}")

# Mass reconstruction using the formula: m = p * sqrt((Ih - C) / K)
mass_t5 = pt_t5[mask_t5] * np.sqrt((Ih_t5[mask_t5] - C) / K)

print(f"Mass range: {mass_t5.min():.3f} - {mass_t5.max():.3f} GeV")

# Create ROOT histogram for inline display with JSROOT
hist_t5 = rt.TH1D("hist_trunc5", "Mass Distribution (Truncated Mean Estimator - DeDx_ItStrip5);Reconstructed Mass [GeV];Number of Tracks", 100, 0, 5000)

# Fill histogram
for mass in mass_t5:
    hist_t5.Fill(mass)

# Style and draw histogram
hist_t5.SetFillColor(rt.kGreen)
hist_t5.SetFillStyle(1001)

# Create canvas and draw histogram
canvas_t5 = rt.TCanvas("canvas_t5", "Truncated Mean 5 Mass Distribution", 800, 600)
hist_t5.Draw("HIST")
canvas_t5.Draw()

print(f"Truncated Mean (DeDx_ItStrip5): {np.sum(mask_t5)} tracks with valid mass reconstruction")
print(f"Mean mass: {mass_t5.mean():.1f} GeV")
print(f"Median mass: {np.median(mass_t5):.1f} GeV")

Data shape - pt: (1252,), Ih: (1252,)
pt range: 9.895 - 458320.549 GeV
Ih range: -1.000 - 21.455
Tracks with Ih > C: 1161 / 1252
Mass range: 2.307 - 501442.611 GeV
Truncated Mean (DeDx_ItStrip5): 1161 tracks with valid mass reconstruction
Mean mass: 2009.4 GeV
Median mass: 1532.3 GeV


In [21]:
# Mass reconstruction for Truncated Mean Estimator (DeDx_ItStrip10)

# Get the truncated mean estimator data
pt_data_t10 = df_pd['IsoTrack_p']
ih_data_t10 = df_pd['DeDx_ItStrip10']

# Convert and flatten the data properly (handle nested arrays)
try:
    pt_t10 = np.concatenate([np.array(track_pt) if hasattr(track_pt, '__iter__') else [track_pt] for track_pt in pt_data_t10])
    Ih_t10 = np.concatenate([np.array(track_ih) if hasattr(track_ih, '__iter__') else [track_ih] for track_ih in ih_data_t10])
except:
    # Fallback to direct conversion if data is already flat
    pt_t10 = np.array(pt_data_t10)
    Ih_t10 = np.array(ih_data_t10)

print(f"Data shape - pt: {pt_t10.shape}, Ih: {Ih_t10.shape}")
print(f"pt range: {pt_t10.min():.3f} - {pt_t10.max():.3f} GeV")
print(f"Ih range: {Ih_t10.min():.3f} - {Ih_t10.max():.3f}")

# Apply mask for valid Ih values (Ih > C)
mask_t10 = Ih_t10 > C
print(f"Tracks with Ih > C: {np.sum(mask_t10)} / {len(Ih_t10)}")

# Mass reconstruction using the formula: m = p * sqrt((Ih - C) / K)
mass_t10 = pt_t10[mask_t10] * np.sqrt((Ih_t10[mask_t10] - C) / K)

print(f"Mass range: {mass_t10.min():.3f} - {mass_t10.max():.3f} GeV")

# Create ROOT histogram for inline display with JSROOT
hist_t10 = rt.TH1D("hist_trunc10", "Mass Distribution (Truncated Mean Estimator - DeDx_ItStrip10);Reconstructed Mass [GeV];Number of Tracks", 100, 0, 5000)

# Fill histogram
for mass in mass_t10:
    hist_t10.Fill(mass)

# Style and draw histogram
hist_t10.SetFillColor(rt.kViolet)
hist_t10.SetFillStyle(1001)

# Create canvas and draw histogram
canvas_t10 = rt.TCanvas("canvas_t10", "Truncated Mean 10 Mass Distribution", 800, 600)
hist_t10.Draw("HIST")
canvas_t10.Draw()

print(f"Truncated Mean (DeDx_ItStrip10): {np.sum(mask_t10)} tracks with valid mass reconstruction")
print(f"Mean mass: {mass_t10.mean():.1f} GeV")
print(f"Median mass: {np.median(mass_t10):.1f} GeV")

Data shape - pt: (1252,), Ih: (1252,)
pt range: 9.895 - 458320.549 GeV
Ih range: -1.000 - 21.455
Tracks with Ih > C: 1147 / 1252
Mass range: 2.307 - 501442.611 GeV
Truncated Mean (DeDx_ItStrip10): 1147 tracks with valid mass reconstruction
Mean mass: 1958.4 GeV
Median mass: 1447.7 GeV


In [22]:
# Mass reconstruction for Truncated Mean Estimator (DeDx_ItStrip15)

# Get the truncated mean estimator data
pt_data_t15 = df_pd['IsoTrack_p']
ih_data_t15 = df_pd['DeDx_ItStrip15']

# Convert and flatten the data properly (handle nested arrays)
try:
    pt_t15 = np.concatenate([np.array(track_pt) if hasattr(track_pt, '__iter__') else [track_pt] for track_pt in pt_data_t15])
    Ih_t15 = np.concatenate([np.array(track_ih) if hasattr(track_ih, '__iter__') else [track_ih] for track_ih in ih_data_t15])
except:
    # Fallback to direct conversion if data is already flat
    pt_t15 = np.array(pt_data_t15)
    Ih_t15 = np.array(ih_data_t15)

print(f"Data shape - pt: {pt_t15.shape}, Ih: {Ih_t15.shape}")
print(f"pt range: {pt_t15.min():.3f} - {pt_t15.max():.3f} GeV")
print(f"Ih range: {Ih_t15.min():.3f} - {Ih_t15.max():.3f}")

# Apply mask for valid Ih values (Ih > C)
mask_t15 = Ih_t15 > C
print(f"Tracks with Ih > C: {np.sum(mask_t15)} / {len(Ih_t15)}")

# Mass reconstruction using the formula: m = p * sqrt((Ih - C) / K)
mass_t15 = pt_t15[mask_t15] * np.sqrt((Ih_t15[mask_t15] - C) / K)

print(f"Mass range: {mass_t15.min():.3f} - {mass_t15.max():.3f} GeV")

# Create ROOT histogram for inline display with JSROOT
hist_t15 = rt.TH1D("hist_trunc15", "Mass Distribution (Truncated Mean Estimator - DeDx_ItStrip15);Reconstructed Mass [GeV];Number of Tracks", 100, 0, 5000)

# Fill histogram
for mass in mass_t15:
    hist_t15.Fill(mass)

# Style and draw histogram
hist_t15.SetFillColor(rt.kGreen)
hist_t15.SetFillStyle(1001)

# Create canvas and draw histogram
canvas_t15 = rt.TCanvas("canvas_t15", "Truncated Mean 15 Mass Distribution", 800, 600)
hist_t15.Draw("HIST")
canvas_t15.Draw()

print(f"Truncated Mean (DeDx_ItStrip15): {np.sum(mask_t15)} tracks with valid mass reconstruction")
print(f"Mean mass: {mass_t15.mean():.1f} GeV")
print(f"Median mass: {np.median(mass_t15):.1f} GeV")

Data shape - pt: (1252,), Ih: (1252,)
pt range: 9.895 - 458320.549 GeV
Ih range: -1.000 - 21.455
Tracks with Ih > C: 1136 / 1252
Mass range: 1.958 - 479287.085 GeV
Truncated Mean (DeDx_ItStrip15): 1136 tracks with valid mass reconstruction
Mean mass: 1929.6 GeV
Median mass: 1430.3 GeV


In [23]:
# Mass reconstruction for Truncated Mean Estimator (DeDx_ItStrip20)

# Get the truncated mean estimator data
pt_data_t20 = df_pd['IsoTrack_p']
ih_data_t20 = df_pd['DeDx_ItStrip20']

# Convert and flatten the data properly (handle nested arrays)
try:
    pt_t20 = np.concatenate([np.array(track_pt) if hasattr(track_pt, '__iter__') else [track_pt] for track_pt in pt_data_t20])
    Ih_t20 = np.concatenate([np.array(track_ih) if hasattr(track_ih, '__iter__') else [track_ih] for track_ih in ih_data_t20])
except:
    # Fallback to direct conversion if data is already flat
    pt_t20 = np.array(pt_data_t20)
    Ih_t20 = np.array(ih_data_t20)

print(f"Data shape - pt: {pt_t20.shape}, Ih: {Ih_t20.shape}")
print(f"pt range: {pt_t20.min():.3f} - {pt_t20.max():.3f} GeV")
print(f"Ih range: {Ih_t20.min():.3f} - {Ih_t20.max():.3f}")

# Apply mask for valid Ih values (Ih > C)
mask_t20 = Ih_t20 > C
print(f"Tracks with Ih > C: {np.sum(mask_t20)} / {len(Ih_t20)}")

# Mass reconstruction using the formula: m = p * sqrt((Ih - C) / K)
mass_t20 = pt_t20[mask_t20] * np.sqrt((Ih_t20[mask_t20] - C) / K)

print(f"Mass range: {mass_t20.min():.3f} - {mass_t20.max():.3f} GeV")

# Create ROOT histogram for inline display with JSROOT
hist_t20 = rt.TH1D("hist_trunc20", "Mass Distribution (Truncated Mean Estimator - DeDx_ItStrip20);Reconstructed Mass [GeV];Number of Tracks", 100, 0, 5000)

# Fill histogram
for mass in mass_t20:
    hist_t20.Fill(mass)

# Style and draw histogram
hist_t20.SetFillColor(rt.kYellow)
hist_t20.SetFillStyle(1001)

# Create canvas and draw histogram
canvas_t20 = rt.TCanvas("canvas_t20", "Truncated Mean 20 Mass Distribution", 800, 600)
hist_t20.Draw("HIST")
canvas_t20.Draw()

print(f"Truncated Mean (DeDx_ItStrip20): {np.sum(mask_t20)} tracks with valid mass reconstruction")
print(f"Mean mass: {mass_t20.mean():.1f} GeV")
print(f"Median mass: {np.median(mass_t20):.1f} GeV")

Data shape - pt: (1252,), Ih: (1252,)
pt range: 9.895 - 458320.549 GeV
Ih range: -1.000 - 20.879
Tracks with Ih > C: 1109 / 1252
Mass range: 2.949 - 479287.085 GeV
Truncated Mean (DeDx_ItStrip20): 1109 tracks with valid mass reconstruction
Mean mass: 1921.8 GeV
Median mass: 1401.8 GeV


In [24]:
# Mass reconstruction for Truncated Mean Estimator (DeDx_ItStrip25)

# Get the truncated mean estimator data
pt_data_t25 = df_pd['IsoTrack_p']
ih_data_t25 = df_pd['DeDx_ItStrip25']

# Convert and flatten the data properly (handle nested arrays)
try:
    pt_t25 = np.concatenate([np.array(track_pt) if hasattr(track_pt, '__iter__') else [track_pt] for track_pt in pt_data_t25])
    Ih_t25 = np.concatenate([np.array(track_ih) if hasattr(track_ih, '__iter__') else [track_ih] for track_ih in ih_data_t25])
except:
    # Fallback to direct conversion if data is already flat
    pt_t25 = np.array(pt_data_t25)
    Ih_t25 = np.array(ih_data_t25)

print(f"Data shape - pt: {pt_t25.shape}, Ih: {Ih_t25.shape}")
print(f"pt range: {pt_t25.min():.3f} - {pt_t25.max():.3f} GeV")
print(f"Ih range: {Ih_t25.min():.3f} - {Ih_t25.max():.3f}")

# Apply mask for valid Ih values (Ih > C)
mask_t25 = Ih_t25 > C
print(f"Tracks with Ih > C: {np.sum(mask_t25)} / {len(Ih_t25)}")

# Mass reconstruction using the formula: m = p * sqrt((Ih - C) / K)
mass_t25 = pt_t25[mask_t25] * np.sqrt((Ih_t25[mask_t25] - C) / K)

print(f"Mass range: {mass_t25.min():.3f} - {mass_t25.max():.3f} GeV")

# Create ROOT histogram for inline display with JSROOT
hist_t25 = rt.TH1D("hist_trunc25", "Mass Distribution (Truncated Mean Estimator - DeDx_ItStrip25);Reconstructed Mass [GeV];Number of Tracks", 100, 0, 5000)

# Fill histogram
for mass in mass_t25:
    hist_t25.Fill(mass)

# Style and draw histogram
hist_t25.SetFillColor(rt.kOrange+2)
hist_t25.SetFillStyle(1001)

# Create canvas and draw histogram
canvas_t25 = rt.TCanvas("canvas_t25", "Truncated Mean 25 Mass Distribution", 800, 600)
hist_t25.Draw("HIST")
canvas_t25.Draw()

print(f"Truncated Mean (DeDx_ItStrip25): {np.sum(mask_t25)} tracks with valid mass reconstruction")
print(f"Mean mass: {mass_t25.mean():.1f} GeV")
print(f"Median mass: {np.median(mass_t25):.1f} GeV")

Data shape - pt: (1252,), Ih: (1252,)
pt range: 9.895 - 458320.549 GeV
Ih range: -1.000 - 20.879
Tracks with Ih > C: 1067 / 1252
Mass range: 1.983 - 450802.572 GeV
Truncated Mean (DeDx_ItStrip25): 1067 tracks with valid mass reconstruction
Mean mass: 1935.8 GeV
Median mass: 1394.9 GeV


In [25]:
# Mass reconstruction for Truncated Mean Estimator (DeDx_ItStrip30)

# Get the truncated mean estimator data
pt_data_t30 = df_pd['IsoTrack_p']
ih_data_t30 = df_pd['DeDx_ItStrip30']

# Convert and flatten the data properly (handle nested arrays)
try:
    pt_t30 = np.concatenate([np.array(track_pt) if hasattr(track_pt, '__iter__') else [track_pt] for track_pt in pt_data_t30])
    Ih_t30 = np.concatenate([np.array(track_ih) if hasattr(track_ih, '__iter__') else [track_ih] for track_ih in ih_data_t30])
except:
    # Fallback to direct conversion if data is already flat
    pt_t30 = np.array(pt_data_t30)
    Ih_t30 = np.array(ih_data_t30)

print(f"Data shape - pt: {pt_t30.shape}, Ih: {Ih_t30.shape}")
print(f"pt range: {pt_t30.min():.3f} - {pt_t30.max():.3f} GeV")
print(f"Ih range: {Ih_t30.min():.3f} - {Ih_t30.max():.3f}")

# Apply mask for valid Ih values (Ih > C)
mask_t30 = Ih_t30 > C
print(f"Tracks with Ih > C: {np.sum(mask_t30)} / {len(Ih_t30)}")

# Mass reconstruction using the formula: m = p * sqrt((Ih - C) / K)
mass_t30 = pt_t30[mask_t30] * np.sqrt((Ih_t30[mask_t30] - C) / K)

print(f"Mass range: {mass_t30.min():.3f} - {mass_t30.max():.3f} GeV")

# Create ROOT histogram for inline display with JSROOT
hist_t30 = rt.TH1D("hist_trunc30", "Mass Distribution (Truncated Mean Estimator - DeDx_ItStrip30);Reconstructed Mass [GeV];Number of Tracks", 100, 0, 5000)

# Fill histogram
for mass in mass_t30:
    hist_t30.Fill(mass)

# Style and draw histogram
hist_t30.SetFillColor(rt.kRed+2)
hist_t30.SetFillStyle(1001)

# Create canvas and draw histogram
canvas_t30 = rt.TCanvas("canvas_t30", "Truncated Mean 30 Mass Distribution", 800, 600)
hist_t30.Draw("HIST")
canvas_t30.Draw()

print(f"Truncated Mean (DeDx_ItStrip30): {np.sum(mask_t30)} tracks with valid mass reconstruction")
print(f"Mean mass: {mass_t30.mean():.1f} GeV")
print(f"Median mass: {np.median(mass_t30):.1f} GeV")

Data shape - pt: (1252,), Ih: (1252,)
pt range: 9.895 - 458320.549 GeV
Ih range: -1.000 - 20.879
Tracks with Ih > C: 1036 / 1252
Mass range: 0.996 - 450802.572 GeV
Truncated Mean (DeDx_ItStrip30): 1036 tracks with valid mass reconstruction
Mean mass: 1960.9 GeV
Median mass: 1391.5 GeV


In [26]:
# Mass reconstruction for Truncated Mean Estimator (DeDx_ItStrip35)

# Get the truncated mean estimator data
pt_data_t35 = df_pd['IsoTrack_p']
ih_data_t35 = df_pd['DeDx_ItStrip35']

# Convert and flatten the data properly (handle nested arrays)
try:
    pt_t35 = np.concatenate([np.array(track_pt) if hasattr(track_pt, '__iter__') else [track_pt] for track_pt in pt_data_t35])
    Ih_t35 = np.concatenate([np.array(track_ih) if hasattr(track_ih, '__iter__') else [track_ih] for track_ih in ih_data_t35])
except:
    # Fallback to direct conversion if data is already flat
    pt_t35 = np.array(pt_data_t35)
    Ih_t35 = np.array(ih_data_t35)

print(f"Data shape - pt: {pt_t35.shape}, Ih: {Ih_t35.shape}")
print(f"pt range: {pt_t35.min():.3f} - {pt_t35.max():.3f} GeV")
print(f"Ih range: {Ih_t35.min():.3f} - {Ih_t35.max():.3f}")

# Apply mask for valid Ih values (Ih > C)
mask_t35 = Ih_t35 > C
print(f"Tracks with Ih > C: {np.sum(mask_t35)} / {len(Ih_t35)}")

# Mass reconstruction using the formula: m = p * sqrt((Ih - C) / K)
mass_t35 = pt_t35[mask_t35] * np.sqrt((Ih_t35[mask_t35] - C) / K)

print(f"Mass range: {mass_t35.min():.3f} - {mass_t35.max():.3f} GeV")

# Create ROOT histogram for inline display with JSROOT
hist_t35 = rt.TH1D("hist_trunc35", "Mass Distribution (Truncated Mean Estimator - DeDx_ItStrip35);Reconstructed Mass [GeV];Number of Tracks", 100, 0, 5000)

# Fill histogram
for mass in mass_t35:
    hist_t35.Fill(mass)

# Style and draw histogram
hist_t35.SetFillColor(rt.kPink)
hist_t35.SetFillStyle(1001)

# Create canvas and draw histogram
canvas_t35 = rt.TCanvas("canvas_t35", "Truncated Mean 35 Mass Distribution", 800, 600)
hist_t35.Draw("HIST")
canvas_t35.Draw()

print(f"Truncated Mean (DeDx_ItStrip35): {np.sum(mask_t35)} tracks with valid mass reconstruction")
print(f"Mean mass: {mass_t35.mean():.1f} GeV")
print(f"Median mass: {np.median(mass_t35):.1f} GeV")

Data shape - pt: (1252,), Ih: (1252,)
pt range: 9.895 - 458320.549 GeV
Ih range: -1.000 - 20.879
Tracks with Ih > C: 1008 / 1252
Mass range: 0.996 - 437435.885 GeV
Truncated Mean (DeDx_ItStrip35): 1008 tracks with valid mass reconstruction
Mean mass: 1974.3 GeV
Median mass: 1379.4 GeV


## Fit Estimator (Landau MPV)

In [27]:

# Get cluster data for fitting
cluster = df_filtered.AsNumpy(["cluster_DeDxStrip"])["cluster_DeDxStrip"]

# Use the new fit function that directly returns MPV-pt pairs
print("\nPerforming Landau fits... this may take a moment")
fit_results = utils.fit(cluster, threshold=2, max_hists=10000)

# Extract MPV-pt pairs - handle inhomogeneous pt structure
mpv_pt_pairs = fit_results["mpv_pt_pairs"]  # List of (mpv, pt) tuples

print(f"Got {len(mpv_pt_pairs)} MPV-pt pairs")

# Debug: Check structure of first few pairs
print("\nDebugging pair structure:")
for i in range(min(3, len(mpv_pt_pairs))):
    mpv, pt = mpv_pt_pairs[i]
    print(f"  Pair {i}: MPV type={type(mpv)}, MPV={mpv}, pt type={type(pt)}, pt shape={np.array(pt).shape if hasattr(pt, '__len__') else 'scalar'}")

# Extract MPV values (should be straightforward)
fit_mpvs = np.array([pair[0] for pair in mpv_pt_pairs])

# Extract pt values - handle the inhomogeneous structure
fit_pt_values = []
for pair in mpv_pt_pairs:
    pt_val = pair[1]
    # If pt_val is array-like, take the first element; if scalar, use as-is
    if hasattr(pt_val, '__len__') and not isinstance(pt_val, str):
        fit_pt_values.append(pt_val[0] if len(pt_val) > 0 else 0.0)
    else:
        fit_pt_values.append(float(pt_val))

fit_pt_values = np.array(fit_pt_values)

print(f"Successfully fitted {len(fit_mpvs)} tracks")
print(f"MPV array shape: {fit_mpvs.shape}")
print(f"pt array shape: {fit_pt_values.shape}")
print(f"Perfect pairing: {len(fit_mpvs) == len(fit_pt_values)}")

# Show some example pairs to verify
print(f"\nFirst 5 MPV-pt pairs:")
for i in range(min(5, len(mpv_pt_pairs))):
    mpv = fit_mpvs[i]
    pt = fit_pt_values[i]
    print(f"  Track {i}: MPV = {mpv:.3f}, pt = {pt:.3f} GeV")


Performing Landau fits... this may take a moment
Got 1167 MPV-pt pairs

Debugging pair structure:
  Pair 0: MPV type=<class 'float'>, MPV=5.799880337726194, pt type=<class 'float'>, pt shape=scalar
  Pair 1: MPV type=<class 'float'>, MPV=2.9740491742618045, pt type=<class cppyy.gbl.std.vector<double> at 0x50365490>, pt shape=(1,)
  Pair 2: MPV type=<class 'float'>, MPV=5.442898320287949, pt type=<class 'float'>, pt shape=scalar
Successfully fitted 1167 tracks
MPV array shape: (1167,)
pt array shape: (1167,)
Perfect pairing: True

First 5 MPV-pt pairs:
  Track 0: MPV = 5.800, pt = 1439.185 GeV
  Track 1: MPV = 2.974, pt = 761.051 GeV
  Track 2: MPV = 5.443, pt = 1157.570 GeV
  Track 3: MPV = 5.073, pt = 761.051 GeV
  Track 4: MPV = 6.672, pt = 986.147 GeV


Error in <Minuit2>: VariableMetricBuilder Initial matrix not pos.def.
Info in <ROOT::Math::ParameterSettings>: lower/upper bounds outside current parameter value. The value will be set to (low+up)/2 
Error in <Minuit2>: VariableMetricBuilder Initial matrix not pos.def.
Info in <ROOT::Math::ParameterSettings>: lower/upper bounds outside current parameter value. The value will be set to (low+up)/2 
Info in <ROOT::Math::ParameterSettings>: lower/upper bounds outside current parameter value. The value will be set to (low+up)/2 
Info in <ROOT::Math::ParameterSettings>: lower/upper bounds outside current parameter value. The value will be set to (low+up)/2 
Info in <ROOT::Math::ParameterSettings>: lower/upper bounds outside current parameter value. The value will be set to (low+up)/2 
Error in <Minuit2>: VariableMetricBuilder Initial matrix not pos.def.
Error in <Minuit2>: VariableMetricBuilder Initial matrix not pos.def.
Info in <ROOT::Math::ParameterSettings>: lower/upper bounds outside cu

In [29]:
# Mass reconstruction using the perfectly paired MPV-pt data

print("=== Mass Reconstruction with Fit Estimator ===")


print(f"Working with {len(fit_mpvs)} MPV-p pairs")

# Mass reconstruction using the formula: m = p * sqrt((mpv - C) / K)
mask_fit = fit_mpvs > C
mass_fit = np.zeros_like(fit_pt_values)
mass_fit[mask_fit] = fit_pt_values[mask_fit] * np.sqrt((fit_mpvs[mask_fit] - C) / K)

print(f"\nMass reconstruction results:")
print(f"Tracks with MPV > C: {np.sum(mask_fit)} / {len(fit_mpvs)}")
print(f"Mass range: {mass_fit[mask_fit].min():.3f} - {mass_fit[mask_fit].max():.3f} GeV")

# Create ROOT histogram for inline display with JSROOT
hist_fit = rt.TH1D("hist_fit", "Mass Distribution (Fit Estimator - Landau MPV);Reconstructed Mass [GeV];Number of Tracks", 100, 0, 5000)

# Fill histogram
for mass in mass_fit[mask_fit]:
    hist_fit.Fill(mass)

# Style and draw histogram
hist_fit.SetFillColor(rt.kViolet+2)
hist_fit.SetFillStyle(1001)

# Create canvas and draw histogram
canvas_fit = rt.TCanvas("canvas_fit", "Fit Estimator Mass Distribution", 800, 600)
hist_fit.Draw("HIST")
canvas_fit.Draw()
print(f"MPV range: {np.min(fit_mpvs):.3f} - {np.max(fit_mpvs):.3f}")
print(f"Fit Estimator (Landau MPV): {len(mass_fit[mask_fit])} tracks with valid mass reconstruction")
print(f"Mean mass: {mass_fit[mask_fit].mean():.1f} GeV")
print(f"Median mass: {np.median(mass_fit[mask_fit]):.1f} GeV")
print(f"Average MPV: {np.mean(fit_mpvs):.3f}")


# Verification: Show perfect correspondence
print(f"\nVerification - First 5 MPV-pt-mass triplets:")
for i in range(min(5, len(fit_mpvs))):
    if mask_fit[i]:  # Only show tracks with valid mass
        print(f"  Track {i}: MPV = {fit_mpvs[i]:.3f}, pt = {fit_pt_values[i]:.3f} GeV, mass = {mass_fit[i]:.1f} GeV")

=== Mass Reconstruction with Fit Estimator ===
Working with 1167 MPV-p pairs

Mass reconstruction results:
Tracks with MPV > C: 984 / 1167
Mass range: 3.667 - 137744.627 GeV
MPV range: -6.872 - 19.657
Fit Estimator (Landau MPV): 984 tracks with valid mass reconstruction
Mean mass: 1074.9 GeV
Median mass: 723.6 GeV
Average MPV: 5.360

Verification - First 5 MPV-pt-mass triplets:
  Track 0: MPV = 5.800, pt = 1439.185 GeV, mass = 1476.2 GeV
  Track 2: MPV = 5.443, pt = 1157.570 GeV, mass = 1103.5 GeV
  Track 3: MPV = 5.073, pt = 761.051 GeV, mass = 663.6 GeV
  Track 4: MPV = 6.672, pt = 986.147 GeV, mass = 1167.8 GeV


Warning in <TFile::Append>: Replacing existing TH1: hist_fit (Potential memory leak).
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas_fit
